# cisQTL analysis workflows
Taking the output from data preprocessing stage, this tutorial document the ways to generate cis association scan summary statistics.

There are two modules can be used: TensorQTL, APEX, they both produce the same set of results

The output is a list of sumstat, which can be merged into a sumstat table to feed into multivariate analysis

## APEX

In [ ]:
nohup sos run /home/hs3163/GIT/xqtl-pipeline/pipeline/association_scan/APEX/APEX.ipynb APEX_cis \
--genotype_file_list /mnt/mfs/statgen/xqtl_workflow_testing/module/genotype_format/Dry_vcf_geno/Dry.vcf_geno_list.txt \
--molecular_pheno_list /mnt/mfs/statgen/xqtl_workflow_testing/module/phenotype/Dry.processed_phenotype.per_chrom.recipe  \
--covariate /mnt/mfs/statgen/xqtl_workflow_testing/success_example/testing_no_cov/Data_Processing/Factor_and_Covariate/AC.APEX.cov   \
--grm_list /mnt/mfs/statgen/xqtl_workflow_testing/module/grm/Dry.grm_list.txt  \
--wd ./  \
--name "demo" --container "/mnt/mfs/statgen/containers/xqtl_pipeline_sif/apex.sif" &

## TensorQTL

In [ ]:
nohup sos run /home/hs3163/GIT/xqtl-pipeline/pipeline/association_scan/TensorQTL/TensorQTL.ipynb TensorQTL_cis \
--genotype_file_list /mnt/mfs/statgen/xqtl_workflow_testing/demo/GWAS_QC/demo.processed_genotype.plink_per_chrom.recipe \
--molecular_pheno_list /mnt/mfs/statgen/xqtl_workflow_testing/demo/phenotype_reformat/demo.processed_phenotype.per_chrom.recipe  \
--covariate /mnt/mfs/statgen/xqtl_workflow_testing/success_example/testing_no_cov/Data_Processing/Factor_and_Covariate/AC.APEX.cov   \
--wd ./  \
--name "demo" --container "/mnt/mfs/statgen/containers/xqtl_pipeline_sif/TensorQTL.sif" &

In [ ]:
nohup sos dryrun /home/hs3163/GIT/xqtl-pipeline/pipeline/association_scan/TensorQTL/TensorQTL.ipynb TensorQTL_cis \
--genotype_file_list /mnt/mfs/statgen/xqtl_workflow_testing/demo/GWAS_QC/demo.processed_genotype.plink_per_chrom.recipe \
--molecular_pheno_list /mnt/mfs/statgen/xqtl_workflow_testing/demo/phenotype_reformat/demo.processed_phenotype.per_chrom.recipe  \
--covariate /mnt/mfs/statgen/xqtl_workflow_testing/success_example/testing_no_cov/Data_Processing/Factor_and_Covariate/AC.APEX.cov   \
--wd ./  \
--name "demo" --container "/mnt/mfs/statgen/containers/xqtl_pipeline_sif/TensorQTL.sif" &

In [1]:
import pandas as pd
import numpy as np
import tensorqtl
from tensorqtl import genotypeio, cis, trans


In [3]:
## Defineing parameter
plink_prefix_path = '/mnt/mfs/statgen/xqtl_workflow_testing/demo/GWAS_QC_arch/chr6.demo.filtered.prune'
Prefix = "/mnt/mfs/statgen/xqtl_workflow_testing/ROSMAP_test_merger/demo.chr6"
## Loading Data
### Filter by the optional keep gene
##if ..is_file():
##    region = pd.read_csv(".","\t")
##    keep_gene = region["gene_ID"].to_list()
##    phenotype_df = phenotype_df.query('gene_ID  in keep_gene')
##    phenotype_pos_df = phenotype_pos_df.query('gene_ID  in keep_gene')

pr = genotypeio.PlinkReader(plink_prefix_path)
genotype_df = pr.load_genotypes()
variant_df = pr.bim.set_index('snp')[['chrom', 'pos']]

Mapping files: 100%|██████████| 3/3 [00:01<00:00,  2.77it/s]


In [11]:
type(variant_df)
type(genotype_df)

pandas.core.frame.DataFrame

In [9]:
variant_df

,chrom,pos
snp,,
6:162484_A_AT,6,162484
6:203397_C_T,6,203397
6:205213_C_T,6,205213
6:205878_A_G,6,205878
6:206599_C_T,6,206599
...,...,...
6:171045879_A_G,6,171045879
6:171046050_A_G,6,171046050
6:171048873_A_G,6,171048873


In [10]:
genotype_df

iid,MAP15387421,MAP22868024,MAP26637867,MAP29629849,MAP33332646,MAP34726040,MAP46246604,MAP46251007,MAP50103679,MAP50104134,...,SM-CTEN7,SM-CTEN8,SM-CTEN9,SM-CTENA,SM-CTENC,SM-CTEND,SM-CTENE,SM-CTENF,SM-CTENG,SM-CTENH
snp,,,,,,,,,,,,,,,,,,,,,
6:162484_A_AT,1,1,0,0,1,0,0,0,0,0,...,0,2,2,1,0,1,1,1,0,1
6:203397_C_T,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
6:205213_C_T,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6:205878_A_G,1,0,0,1,0,0,1,0,0,0,...,0,0,2,0,0,0,0,0,0,0
6:206599_C_T,0,0,0,2,0,0,1,1,1,1,...,1,1,2,0,0,2,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6:171045879_A_G,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6:171046050_A_G,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6:171048873_A_G,0,0,0,0,0,0,0,1,0,1,...,0,0,1,0,0,0,0,0,1,0
